<a href="https://colab.research.google.com/github/HiruRathnayake/spark_table_manipulation_01/blob/main/spark_table_manipulation_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 35.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=f28aa61e25b0288bdb0e196fbaac85c53113b4c2e93c67d8161f069c7d7581e9
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
#Create DataFrame df1 with columns name,dept & age
import pandas as pd
from pyspark.sql import Row

df1 = spark.createDataFrame([
    Row(name="Janaka", dept="Sales", age=34),
    Row(name="Malaka", dept="Sales", age=30),
    Row(name="Naduni", dept="Sales", age=56),
    Row(name="Sarala", dept="Finance", age=24)
])
df1
df1.printSchema()

root
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- age: long (nullable = true)



In [ ]:
#Create DataFrame df2 with columns name,dep,status & salary
df2 = spark.createDataFrame([
    Row(name=" Janaka ", dept="Sales", status="Single", salary=9000),
    Row(name="Sagara", dept="Finance", status =" Single ", salary=7900),
    Row(name=" Sarala", dept="Finance", status ="Married", salary=9000),
    Row(name="Saman", dept="Marketing", status =" Married ", salary=8000)
])
df2
df2.printSchema()

root
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- status: string (nullable = true)
 |-- salary: long (nullable = true)



In [ ]:
#Add missing columns 'state' & 'salary' to df1
from pyspark.sql.functions import lit
for column in [column for column in df2.columns if column not in df1.columns]:
    df1 = df1.withColumn(column, lit(None))

In [ ]:
#Add missing column 'age' to df2
for column in [column for column in df1.columns if column not in df2.columns]:
    df2 = df2.withColumn(column, lit(None))

In [ ]:
#Merge Two Data Frames 
merged_df=df1.unionByName(df2)
merged_df.show()

+--------+---------+----+---------+------+
|    name|     dept| age|   status|salary|
+--------+---------+----+---------+------+
|  Janaka|    Sales|  34|     null|  null|
|  Malaka|    Sales|  30|     null|  null|
|  Naduni|    Sales|  56|     null|  null|
|  Sarala|  Finance|  24|     null|  null|
| Janaka |    Sales|null|   Single|  9000|
|  Sagara|  Finance|null|  Single |  7900|
|  Sarala|  Finance|null|  Married|  9000|
|   Saman|Marketing|null| Married |  8000|
+--------+---------+----+---------+------+



In [ ]:
#import dataset
import pyspark
from pyspark.sql.session import SparkSession
spark = spark.builder.getOrCreate()
path = "/content/gdrive/My Drive/dataset/sales.csv"
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(path)

In [ ]:
# Register the DataFrame as a global temporary view
df.createGlobalTempView("salesTemp")
spark.sql("SELECT * FROM global_temp.salesTemp").show()

+-------------------+-----------------+-----+-------------+------+--------+--------------------+---------+--------------------+-----+--------------------+----+----------+--------------+---------+----------+------+
|               date|convenience_store|store|county_number|county|category|       category_name|vendor_no|              vendor| item|         description|pack|liter_size|state_btl_cost|btl_price|bottle_qty| total|
+-------------------+-----------------+-----+-------------+------+--------+--------------------+---------+--------------------+-----+--------------------+----+----------+--------------+---------+----------+------+
|2014-11-13 00:00:00|             null| 2635|           82| Scott| 1032200|IMPORTED VODKA - ...|      370|Pernod Ricard USA...|34036|       Absolut Pears|  12|       750|        $11.49|   $17.24|         4| 68.96|
|2014-11-13 00:00:00|             null| 2635|           82| Scott| 1032200|IMPORTED VODKA - ...|      370|Pernod Ricard USA...|34076|     Absolu

In [ ]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- convenience_store: string (nullable = true)
 |-- store: integer (nullable = true)
 |-- county_number: integer (nullable = true)
 |-- county: string (nullable = true)
 |-- category: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- vendor_no: integer (nullable = true)
 |-- vendor: string (nullable = true)
 |-- item: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- pack: integer (nullable = true)
 |-- liter_size: integer (nullable = true)
 |-- state_btl_cost: string (nullable = true)
 |-- btl_price: string (nullable = true)
 |-- bottle_qty: integer (nullable = true)
 |-- total: double (nullable = true)



In [ ]:
#Update the column name
df = df.withColumnRenamed('county_number', 'county_no')
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- convenience_store: string (nullable = true)
 |-- store: integer (nullable = true)
 |-- county_no: integer (nullable = true)
 |-- county: string (nullable = true)
 |-- category: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- vendor_no: integer (nullable = true)
 |-- vendor: string (nullable = true)
 |-- item: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- pack: integer (nullable = true)
 |-- liter_size: integer (nullable = true)
 |-- state_btl_cost: string (nullable = true)
 |-- btl_price: string (nullable = true)
 |-- bottle_qty: integer (nullable = true)
 |-- total: double (nullable = true)



In [ ]:
#Update multiple column names
df = df.withColumnRenamed("county_no","CountryNo").withColumnRenamed("liter_size","LiterSize")
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- convenience_store: string (nullable = true)
 |-- store: integer (nullable = true)
 |-- CountryNo: integer (nullable = true)
 |-- county: string (nullable = true)
 |-- category: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- vendor_no: integer (nullable = true)
 |-- vendor: string (nullable = true)
 |-- item: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- pack: integer (nullable = true)
 |-- LiterSize: integer (nullable = true)
 |-- state_btl_cost: string (nullable = true)
 |-- btl_price: string (nullable = true)
 |-- bottle_qty: integer (nullable = true)
 |-- total: double (nullable = true)



In [ ]:
df.select("pack").show(5)

+----+
|pack|
+----+
|  12|
|  12|
|  12|
|   6|
|  24|
+----+
only showing top 5 rows



In [ ]:
#Update values of an Existing Column
df = df.withColumn("pack",(df.pack)*100)
df.select("pack").show(5)

+----+
|pack|
+----+
|1200|
|1200|
|1200|
| 600|
|2400|
+----+
only showing top 5 rows



In [ ]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- convenience_store: string (nullable = true)
 |-- store: integer (nullable = true)
 |-- CountryNo: integer (nullable = true)
 |-- county: string (nullable = true)
 |-- category: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- vendor_no: integer (nullable = true)
 |-- vendor: string (nullable = true)
 |-- item: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- pack: integer (nullable = true)
 |-- LiterSize: integer (nullable = true)
 |-- state_btl_cost: string (nullable = true)
 |-- btl_price: string (nullable = true)
 |-- bottle_qty: integer (nullable = true)
 |-- total: double (nullable = true)



In [ ]:
#Update the Column Data Type
df = df.withColumn("convenience_store",(df.convenience_store).cast("Integer"))
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- convenience_store: integer (nullable = true)
 |-- store: integer (nullable = true)
 |-- CountryNo: integer (nullable = true)
 |-- county: string (nullable = true)
 |-- category: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- vendor_no: integer (nullable = true)
 |-- vendor: string (nullable = true)
 |-- item: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- pack: integer (nullable = true)
 |-- LiterSize: integer (nullable = true)
 |-- state_btl_cost: string (nullable = true)
 |-- btl_price: string (nullable = true)
 |-- bottle_qty: integer (nullable = true)
 |-- total: double (nullable = true)



In [ ]:
#Create new column by using data from existing column
df = df.withColumn('pack_liter_size',df.pack * df.LiterSize)
df.select("pack","LiterSize","pack_liter_size").show(5)

+----+---------+---------------+
|pack|LiterSize|pack_liter_size|
+----+---------+---------------+
|1200|      750|         900000|
|1200|      750|         900000|
|1200|      750|         900000|
| 600|      750|         450000|
|2400|      200|         480000|
+----+---------+---------------+
only showing top 5 rows



In [ ]:
#Split Column
split_col = pyspark.sql.functions.split(df['date'], '-')
df = df.withColumn('Year', split_col.getItem(0))
df = df.withColumn('Month', split_col.getItem(1))
df.select("date","Year","Month").show(5)

+-------------------+----+-----+
|               date|Year|Month|
+-------------------+----+-----+
|2014-11-13 00:00:00|2014|   11|
|2014-11-13 00:00:00|2014|   11|
|2014-11-13 00:00:00|2014|   11|
|2014-11-13 00:00:00|2014|   11|
|2014-11-13 00:00:00|2014|   11|
+-------------------+----+-----+
only showing top 5 rows



In [ ]:
#Drop a column
df = df.drop("pack_liter_size")
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- convenience_store: string (nullable = true)
 |-- store: integer (nullable = true)
 |-- county_number: integer (nullable = true)
 |-- county: string (nullable = true)
 |-- category: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- vendor_no: integer (nullable = true)
 |-- vendor: string (nullable = true)
 |-- item: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- pack: integer (nullable = true)
 |-- liter_size: integer (nullable = true)
 |-- state_btl_cost: string (nullable = true)
 |-- btl_price: string (nullable = true)
 |-- bottle_qty: integer (nullable = true)
 |-- total: double (nullable = true)

